In [1]:
import polars as pl
from huggingface_hub import snapshot_download
from datasets import load_dataset
import os
import sys
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
sys.path.append("..")
%load_ext autoreload
%autoreload 2


/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
gutenberg_books = pl.read_parquet("../dataset_files/gutenberg_books.parquet", n_rows=10)

In [5]:
gutenberg_books

id,text
i64,str
76645,"""﻿The Project Gutenberg eBook o…"
76644,"""﻿The Project Gutenberg eBook o…"
76604,"""﻿The Project Gutenberg eBook o…"
76596,"""﻿The Project Gutenberg eBook o…"
76660,"""﻿The Project Gutenberg eBook o…"
76656,"""﻿The Project Gutenberg eBook o…"
76591,"""﻿The Project Gutenberg eBook o…"
76623,"""﻿The Project Gutenberg eBook o…"
76570,"""﻿The Project Gutenberg eBook o…"
